In [18]:
import os
import re
import shutil
import pandas as pd
from pandas import Timestamp
from tqdm import tqdm
import glob

from pathlib import Path
from src.anexo_a.process_a_attachment import extracao_consulta_bens, extracao_detalhes_contrato, split_por_contrato
from src.utils.normalize_text import DocumentFormatter

In [13]:
PATH_BASE = r"data\Input\DOC_BASE_2014.xlsx"
PATH_TELA_PRETA = r"data\Input\Anexo A"
PATH_OUTPUT = r"data\Output"

# Processar telas pretas

In [ ]:
df_contratos_sem_zeros = pd.read_excel("2018 faltando 2.xlsx")
df_contratos_sem_zeros.head()

In [ ]:
df_contratos_sem_zeros["Contrato"] = df_contratos_sem_zeros["Contrato"].astype(str).str.zfill(7)
df_contratos_sem_zeros

In [ ]:
import os
import shutil

# Caminho base das pastas
pasta_entrega = "Entrega"
pasta_invalidos = "invalidos"

# Garante que a pasta de inválidos existe
os.makedirs(pasta_invalidos, exist_ok=True)

# Lista de contratos como strings (sem zeros à esquerda)
contratos_validos = set(df_contratos_sem_zeros["Contrato"].astype(str))

# Percorre todos os arquivos nas subpastas de Entrega
for root, dirs, files in os.walk(pasta_entrega):
    for nome_arquivo in files:
        caminho_arquivo = os.path.join(root, nome_arquivo)
        # Verifica se algum contrato está no nome do arquivo
        if not any(str(contrato) in nome_arquivo for contrato in contratos_validos):
            # Move para a pasta invalidos (mantendo apenas o nome do arquivo)
            destino = os.path.join(pasta_invalidos, nome_arquivo)
            shutil.move(caminho_arquivo, destino)

In [ ]:
import os
import shutil
import pandas as pd
from tqdm import tqdm

# Caminho do arquivo de contratos a filtrar
arquivo_filtrar = "filtrar.csv"  # ajuste para o nome/caminho correto do arquivo

# Lê a coluna 'Contrato' do arquivo de filtro
df_filtrar = pd.read_csv(arquivo_filtrar, sep=";")  # ajuste o sep se necessário

# Cria a pasta entrega_parcial se não existir
os.makedirs("entrega_parcial", exist_ok=True)
contratos_filtrar = set(df_filtrar["Contrato"].astype(str))

# Pastas de origem e destino
pasta_entrega = "Entrega"
pasta_destino = "entrega_parcial"

for root, dirs, files in os.walk(pasta_entrega):
    for nome_arquivo in files:
        # Verifica se algum contrato está no nome do arquivo
        if any(str(contrato) in nome_arquivo for contrato in contratos_filtrar):
            caminho_origem = os.path.join(root, nome_arquivo)
            # Calcula o caminho relativo para manter a estrutura de pastas
            caminho_relativo = os.path.relpath(root, pasta_entrega)
            destino_dir = os.path.join(pasta_destino, caminho_relativo)
            os.makedirs(destino_dir, exist_ok=True)
            caminho_destino = os.path.join(destino_dir, nome_arquivo)
            shutil.copy2(caminho_origem, caminho_destino)